#52-文本预处理
##本节目录

*   [处理步骤](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=ltSVAkJi-SVF&line=6&uniqifier=1)
*   [读取数据集](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=BE1LFmRneFrp)
*   [词元化](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=FyvsoyYqeMuW&line=6&uniqifier=1)
*   [词表](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=aTVML0Pqeqb1&line=5&uniqifier=1)
*   [整合所有功能](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=cKrAEBT7gplL&line=6&uniqifier=1)




##启用GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


##安装pytorch和d2l

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
!pip3 install d2l==0.14

     |████████████████████████████████| 48 kB 4.7 MB/s 


#文本预处理
##处理步骤

---



1.   将文本作为字符串加载至内存
2.   将字符串拆分为词元（eg.单词和字符）
3.   建立词表，将拆分的词元映射到数字索引
4.   将文本转换为数字索引序列，方便模型操作



In [ ]:
import collections
import re
from d2l import torch as d2l

##读取数据集

In [ ]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '0cb91d09b814ecdc07b50f31f8dcad3e81d6a86d')

#清洗数据
#消除无用文本，转换大小写
def read_time_machine():
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'),'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数：{len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数：3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


##词元化


---



> 文本行列表中的每个文本序列被转化成以词元为单位的词元列表





In [ ]:
#拆分数据为单词或字母
def tokenize(lines,token='word'):
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('error: Unkown type:' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


##词表


---


> 词表(vocabulary):是一个能让词元和数字索引互相映射的字典



> 语料(corpus):是文本中各个词元的出现频率的统计结果



> 根据词元出现频率分配数字索引，同时移除较少的词元降低词表的稀疏程度



> 特殊词元:
1.   未知词元<unk>
2.   填充词元<pad>
3.   序列开始词元<bos>
4.   序列结束词元<eos>









In [ ]:
class Vocab:
    """文本词表"""
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        #按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(),key=lambda x: x[1],reverse=True)
        #未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx,token in enumerate(self.idx_to_token)}
        self.idx_to_token,self.token_to_idx = [],dict()
        print(self.token_to_idx)
        for token,freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self,tokens):
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): #未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    """统计词元的频率"""
    #这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0],list):
        #将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [ ]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

{}
[('the', 0), ('i', 1), ('and', 2), ('of', 3), ('a', 4), ('to', 5), ('was', 6), ('in', 7), ('that', 8), ('my', 9)]


In [ ]:
for i in [0,10]:
    print('token:',tokens[i])
    print('id:',vocab[tokens[i]])

token: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
id: [0, 18, 49, 39, 2182, 2183, 399]
token: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
id: [2185, 2, 24, 1043, 361, 112, 6, 1420, 2, 1044, 0]


##整合所有功能


---



1.   读取清洗后的文件
2.   词元化
3.   建立词表



In [ ]:
def load_corpus_tine_machine(max_tokens=-1):
    """返回数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines,'char')
    vocab = Vocab(tokens)
    """将所有文本行展平到一个列表"""
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0 :
        corpus = corpus[:max_tokens]
    return corpus,vocab

corpus,vocab = d2l.load_corpus_time_machine()
len(corpus),len(vocab)

(171489, 28)